In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

### Loading

In [ ]:
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline, FunctionTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import roc_auc_score
import mlflow
import mlflow.sklearn
import dagshub

dagshub.init(repo_owner='gnada22', repo_name='ml_assignment_2', mlflow=True)

mlflow.set_experiment("AdaBoost_Training")

train_identity = pd.read_csv('/kaggle/input/ieee-fraud-detection/train_identity.csv')
train_transaction = pd.read_csv('/kaggle/input/ieee-fraud-detection/train_transaction.csv')

train = pd.merge(train_transaction, train_identity, on='TransactionID', how='left')

features = [col for col in train.columns if col not in ["TransactionID", "isFraud"]]
X = train[features]
y = train["isFraud"]

X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

### Cleaning

In [ ]:
with mlflow.start_run(run_name="AdaBoost_Cleaning"):
    threshold = 0.7
    missing_ratio = X_train.isnull().mean()
    cols_to_drop = missing_ratio[missing_ratio > threshold].index.tolist()
    X_train.drop(columns=cols_to_drop, inplace=True)
    X_valid.drop(columns=cols_to_drop, inplace=True)
    mlflow.log_metric("initial_missing_ratio", missing_ratio.mean())
    mlflow.log_param("drop_threshold", threshold)
    mlflow.log_param("num_cols_dropped", len(cols_to_drop))

### Feature Engineering

In [ ]:
numerical_cols = X_train.select_dtypes(include=np.number).columns.tolist()
categorical_cols = X_train.select_dtypes(include=['object']).columns.tolist()

with mlflow.start_run(run_name="AdaBoost_Feature_Engineering"):
    def add_transaction_amt_log(X):
        X = X.copy()
        if "TransactionAmt" in X.columns:
            X["TransactionAmt_log"] = np.log1p(X["TransactionAmt"])
        return X

    log_transformer = FunctionTransformer(add_transaction_amt_log, validate=False)

    numerical_transformer = Pipeline([
        ("imputer", SimpleImputer(strategy="median")),
        ("scaler", StandardScaler())
    ])

    categorical_transformer = Pipeline([
        ("imputer", SimpleImputer(strategy="most_frequent")),
        ("encoder", OneHotEncoder(handle_unknown="ignore", sparse_output=False))
    ])

    if "TransactionAmt" in numerical_cols:
        mlflow.log_param("new_features", "TransactionAmt_log")

### Feature Selection

In [ ]:
with mlflow.start_run(run_name="AdaBoost_Feature_Selection"):
    k_best = 50
    selector = SelectKBest(score_func=f_classif, k=k_best)
    selector.fit(X_train[numerical_cols].fillna(0), y_train)
    selected_features = np.array(numerical_cols)[selector.get_support()].tolist()

    if "TransactionAmt" in selected_features:
        selected_features.append("TransactionAmt_log")
    
    mlflow.log_param("num_selected_features", len(selected_features))
    mlflow.log_param("selected_features", ", ".join(selected_features))

### Training

In [ ]:
with mlflow.start_run(run_name="AdaBoost_Model_Training"):
    preprocessor = Pipeline([
        ("log", log_transformer),
        ("column_transform", ColumnTransformer([
            ("num", numerical_transformer, selected_features),
            ("cat", categorical_transformer, categorical_cols)
        ]))
    ])

    ab = AdaBoostClassifier(random_state=42)

    pipeline = Pipeline([
        ("preprocessor", preprocessor),
        ("classifier", ab)
    ])

    param_grid = {
        'classifier__n_estimators': [50, 100, 200],
        'classifier__learning_rate': [0.5, 1.0, 1.5]
    }

    grid_search = GridSearchCV(pipeline, param_grid, scoring='roc_auc', cv=3, n_jobs=-1)
    grid_search.fit(X_train, y_train)

    best_model = grid_search.best_estimator_
    best_params = grid_search.best_params_

    y_train_pred = best_model.predict_proba(X_train)[:, 1]
    y_valid_pred = best_model.predict_proba(X_valid)[:, 1]

    train_auc = roc_auc_score(y_train, y_train_pred)
    valid_auc = roc_auc_score(y_valid, y_valid_pred)

    mlflow.log_metrics({"train_auc": train_auc, "valid_auc": valid_auc})
    mlflow.log_params(best_params)

    mlflow.sklearn.log_model(best_model, "fraud_pipeline")

    print(f"Train AUC: {train_auc:.4f}")
    print(f"Validation AUC: {valid_auc:.4f}")